In [1]:
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.layers import * 
from tensorflow.keras.utils import plot_model, CustomObjectScope
from tensorflow.keras import utils
from tensorflow.keras.datasets import cifar10, mnist
from tensorflow.keras.models import load_model, save_model, Sequential, Model
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow import keras
import numpy as np

import matplotlib.pyplot as plt
import os

In [2]:
class ActivityRegularizationLayer(layers.Layer):
    def __init__(self, rate=1e-2 ,**kwargs):
        super(ActivityRegularizationLayer, self).__init__(**kwargs)
        self.rate = rate
        
    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs
    
    def get_config(self):
        config = super(ActivityRegularizationLayer, self).get_config()
        config.update({"rate":self.rate})
        return config

In [6]:
class OuterLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(OuterLayer, self).__init__(**kwargs)
    
    def call(self, inputs):
        self.activity_reg = ActivityRegularizationLayer(1e-2)
        return self.activity_reg(inputs)
    
    def get_config(self):
        return super(OuterLayer, self).get_config()

In [13]:
class OuterLayerWithKernelRegularizer(layers.Layer):
    def __init__(self, **kwargs):
        super(OuterLayerWithKernelRegularizer, self).__init__(**kwargs)
        self.dense = Dense(32, kernel_regularizer=tf.keras.regularizers.l2(1e-3))
        
    def call(self, inputs):
        return self.dense(inputs)
    
    def get_config(self):
        return super(OuterLayerWithKernelRegularizer, self).get_config()

In [14]:
layer = OuterLayerWithKernelRegularizer()
_ = layer(tf.zeros((1,1)))
print(layer.losses)

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0020365117>]
